In [1]:
from pyspark.sql import SparkSession
import requests
import pyspark
import json

In [2]:
token = ' '
headers = {'Accept':'application/vnd.github+json', 'authorization': f'Bearer {token}'}
max_id = 0

In [3]:
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.getOrCreate()

In [4]:
def get_organizations():
        list_of_organizations = []
        url = 'https://api.github.com/organizations?per_page=100'
        for i in range(2):
            global max_id
            parameter = {'since':max_id}
            response = requests.get(url, params = parameter, headers = headers)
            organizations_rdd = sc.parallelize(response.json())
            list_of_organizations.extend([organization['login'] for organization in organizations_rdd.collect()])
            max_id= max([organization['id'] for organization in organizations_rdd.collect() ])+1
        return list_of_organizations

In [5]:
organizations = get_organizations()

In [6]:
def get_repositories(organizations):
        list_of_repositories =[]
        for organization in organizations:
            link = 'https://api.github.com/orgs/'+ organization +'/repos'
            link_to_the_repositories = requests.get(link, headers = headers)
            repositories_rdd = sc.parallelize(link_to_the_repositories.json())
            list_of_repositories.extend([[repository['owner']['login'], repository['name'], repository['stargazers_count']] for repository in repositories_rdd.collect()])
        return list_of_repositories

In [7]:
repositories = get_repositories(organizations)

In [8]:
Org_and_repos_df = spark.createDataFrame(repositories)
Org_and_repos_df = Org_and_repos_df.withColumnRenamed("_1", "Organization").withColumnRenamed('_2', 'Repository').withColumnRenamed('_2', 'Repository').withColumnRenamed('_3', 'Stars')
Org_and_repos_df.printSchema()

root
 |-- Organization: string (nullable = true)
 |-- Repository: string (nullable = true)
 |-- Stars: long (nullable = true)



In [9]:
Org_and_repos_df.show()

+------------+------------------+-----+
|Organization|        Repository|Stars|
+------------+------------------+-----+
|     errfree|             test1|    2|
|     errfree|              test|    1|
|  engineyard|             eycap|  116|
|  engineyard|           mongrel|   44|
|  engineyard|      vertebra-erl|   34|
|  engineyard|       vertebra-rb|   59|
|  engineyard|           xmpp4em|   13|
|  engineyard|      vertebra-gem|   45|
|  engineyard|      vertebra-xen|   47|
|  engineyard|puppet-daemontools|    6|
|  engineyard|          vertebra|  161|
|  engineyard|            sequel|    2|
|  engineyard|            natter|   76|
|  engineyard|      vertebra-ref|   43|
|  engineyard|   rails-2.2.2-app|   10|
|  engineyard|         loudmouth|   20|
|  engineyard|    loudmouth-ruby|    9|
|  engineyard|          rack-app|    7|
|  engineyard|  ey-cloud-recipes| 1000|
|  engineyard|    oauth_provider|    3|
+------------+------------------+-----+
only showing top 20 rows



In [10]:
Top_repositories = Org_and_repos_df.sort("Stars", ascending=False)
Top_repositories.show()

+--------------+-------------+------+
|  Organization|   Repository| Stars|
+--------------+-------------+------+
|        github|    gitignore|130653|
|         rails|        rails| 50335|
|        github|          hub| 21609|
|       sinatra|      sinatra| 11558|
|       Shopify|      dashing| 11077|
|        github|     linguist|  9622|
|       Shopify|       liquid|  9392|
|    thoughtbot|    paperclip|  9066|
|    thoughtbot|  factory_bot|  7422|
|        github|       markup|  5383|
|        WebKit|  WebKit-http|  5074|
|         rails|         thor|  4829|
|        github|         dmca|  4751|
|collectiveidea|  delayed_job|  4697|
|    thoughtbot|   suspenders|  3769|
|   pluginaweek|state_machine|  3715|
|    thoughtbot|    clearance|  3486|
|      basecamp|          pow|  3442|
|        joyent|        libuv|  3291|
|    thoughtbot| high_voltage|  3094|
+--------------+-------------+------+
only showing top 20 rows



In [11]:
Top_repositories = Top_repositories.toPandas()
Top_20_repositories = Top_repositories[:20]

In [12]:
Top_20_repositories.to_csv('Top 20 repositories by number of stars.csv', index=False)